# Single State Methods for Set Covering problem

In [3]:
import logging
import random
import math
from copy import copy
from collections import Counter
from copy import copy

In [4]:
def problem(N, seed=None):
    """Creates an instance of the problem"""

    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

def goal_test(goal, state):
    return goal == set(_ for _ in state)

## Classical Hill Climbing

In [79]:
def hc(N, all_lists, goal):
    """Vanilla Hill Climber"""
    all_lists = set(tuple(_) for _ in all_lists)
    max_steps = 10_000

    def evaluate(state):
        cnt = Counter()
        cnt.update(sum((e for e in state), start=()))
        return len(cnt), -cnt.total()

    def tweak(solution):
        new_solution = set(solution)
        while new_solution and random.random() < 0.7:
            r = random.choice(list(new_solution))
            new_solution.remove(r)
        while random.random() < 0.7:
            a = random.choice(list(all_lists - solution))
            new_solution.add(a)
        return new_solution

    current_solution = set()
    useless_steps = 0
    while useless_steps < max_steps and not goal_test(goal, current_solution):
        useless_steps += 1
        candidate_solution = tweak(current_solution)
        if evaluate(candidate_solution) > evaluate(current_solution):
            useless_steps = 0
            current_solution = copy(candidate_solution)
            logging.debug(f"New solution: {evaluate(current_solution)}")
    return current_solution

In [80]:
logging.getLogger().setLevel(logging.INFO)

for N in [5, 10, 20, 100]:
    solution = hc(N, problem(N, seed=42), set(range(N)))
    logging.info(
        f" Solution for N={N:,}: "
        + f"w={sum(len(_) for _ in solution):,} "
        + f"(bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)"
    )

INFO:root: Solution for N=5: w=5 (bloat=0%)
INFO:root: Solution for N=10: w=11 (bloat=10%)
INFO:root: Solution for N=20: w=24 (bloat=20%)
INFO:root: Solution for N=100: w=214 (bloat=114%)


## Simulated Annealing

In [81]:
def hc_simulated_annealing(N, all_lists, goal):
    all_lists = set(tuple(_) for _ in all_lists)
    max_steps = 10_000
    
    def calculate_temperature(max_steps, current_step):
        # Define a way to set a certain iteration over the solution space "hot" or "cold"
        return 1-((current_step+1)/max_steps)

    def energy_function(state):
        cnt = Counter()
        cnt.update(sum((e for e in state), start=()))
        return len(cnt), -cnt.total()

    def probability_function(energy1: tuple, energy2: tuple, temperature):
        # Calculate the probability by using energies and temperature
        # Since the energy_function provides us tuples, I normalize the terms for doing an average between them
        # I added one to the denominator to avoid division by 0
        energy1_0 = (energy1[0]*100)/(abs(energy1[0]-energy2[0])+1) 
        energy2_0 = (energy2[0]*100)/(abs(energy1[0]-energy2[0])+1)
        energy1_1 = (energy1[1]*100)/(abs(energy1[1]-energy2[1])+1)
        energy2_1 = (energy2[1]*100)/(abs(energy1[1]-energy2[1])+1)
        first_term = (energy1_0 - energy2_0)
        second_term = (energy1_1 - energy2_1)
        avg = (first_term + second_term)/2
        return math.exp(-avg/temperature)

    def tweak(solution):
        counter = 0
        new_solution = set(solution)
        while new_solution and random.random() < 0.7:
            r = random.choice(list(new_solution))
            new_solution.remove(r)
        while list(all_lists-solution) and random.random() < 0.7:
            # Added the first condition of non-empty list because everytime the random.choice function raises an error
            # I don't know why it happens only with this case and not also in the others
            a = random.choice(list(all_lists - solution))
            new_solution.add(a)
        return new_solution

    current_solution = set()
    useless_steps = 0
    while useless_steps < max_steps and not goal_test(goal, current_solution):
        useless_steps += 1
        temperature = calculate_temperature(max_steps, useless_steps)
        candidate_solution = tweak(current_solution)
        e1 = energy_function(candidate_solution)
        e2 = energy_function(current_solution)
        if probability_function(e1, e2, temperature) > random.random():
            useless_steps = 0
            current_solution = copy(candidate_solution)
            logging.debug(f"New solution: {energy_function(current_solution)}")
    return current_solution

In [82]:
logging.getLogger().setLevel(logging.INFO)

for N in [5, 10, 20, 100]:
    solution = hc_simulated_annealing(N, problem(N, seed=42), set(range(N)))
    logging.info(
        f" Solution for N={N:,}: "
        + f"w={sum(len(_) for _ in solution):,} "
        + f"(bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)"
    )

KeyboardInterrupt: 

## (1+1)-ES (Evolution Strategy)

In [13]:
def one_plus_one_ES(N, all_lists, goal):
    all_lists = set(tuple(_) for _ in all_lists)
    max_steps = 10_000
    sigma = 6.0

    def evaluate(state):
        cnt = Counter()
        cnt.update(sum((e for e in state), start=()))
        return len(cnt), -cnt.total()

    def tweak(solution):
        new_solution = set(solution)
        while new_solution and random.random() < 0.7:
            normal = round(random.normalvariate(0, sigma))
            r = random.choice(list(new_solution))
            index = list(new_solution).index(r)
            new_solution.remove(r)
            print(normal)
            new_index = index + normal
            # Some controls to avoid accesses not allowed in memory
            if new_index >= len(all_lists):
                new_index = len(all_lists)-1
            elif new_index < 0:
                new_index = 0
            if list(all_lists)[new_index] not in solution:
                # If we found a new set to add
                new_solution.add(r)
                break
            else:
                # Try again
                new_solution.add(r)
        while random.random() < 0.7:
            a = random.choice(list(all_lists - solution))
            new_solution.add(a)
        return new_solution

    current_solution = set()
    useless_steps = 0
    while useless_steps < max_steps and not goal_test(goal, current_solution):
        useless_steps += 1
        candidate_solution = tweak(current_solution) 
        if evaluate(candidate_solution) > evaluate(current_solution):
            useless_steps = 0
            current_solution = copy(candidate_solution)
            logging.debug(f"New solution: {evaluate(current_solution)}")
    return current_solution

In [14]:
logging.getLogger().setLevel(logging.INFO)

for N in [5, 10, 20, 100]:
    solution = one_plus_one_ES(N, problem(N, seed=42), set(range(N)))
    logging.info(
        f" Solution for N={N:,}: "
        + f"w={sum(len(_) for _ in solution):,} "
        + f"(bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)"
    )

0
0
0
0
-1
0
0
0
0
0
0
0
0
-1
0
0
0
-1
1
0
0
-1
0
0
0
0
0
0
1
0
0
0
1
-1
0
-1
1
0
-1
0
0
1
-1
0
1
0
-1
1
0
1
0
-1
0
1
0
1
0
0
-1
1
0
0
-1
0
0
0
-1
0
1
0
0
-1
0
1
0
0
0
0
1
0
0
0
0
1
0
-1
-1
0
0
-1
0
0
-1
0
-1
1
1
-1
0
-1
0
1
0
0
0
0
0
0
0
0
0
-1
0
0
1
0
1
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
-1
0
-1
0
-1
-1
1
-1
1
0
0
1
1
1
0
0
0
0
1
0
0
1
1
0
0
-1
0
0
1
-1
0
0
-1
0
1
1
0
0
0
0
0
0
0
-1
0
0
-1
0
0
0
-1
0
1
0
1
1
1
0
0
0
0
0
0
0
-1
0
0
0
-1
0
0
0
0
0
0
0
-1
0
0
1
0
0
0
-1
0
0
0
0
-1
-1
0
0
1
0
0
0
0
0
0
1
1
0
0
1
0
0
0
0
0
1
0
-1
0
0
1
0
0
0
0
0
0
1
-1
0
1
-1
-1
0
0
1
1
0
1
0
0
0
-1
0
0
1
1
-1
-1
0
0
0
0
0
0
0
0
0
0
0
1
1
0
-1
0
0
-1
0
0
-1
0
0
0
0
0
0
1
0
0
1
1
-1
0
0
0
0
0
0
0
0
0
-1
-1
-1
0
0
0
1
0
1
0
0
0
0
1
0
1
0
0
0
0
0
0
0
1
-1
0
1
0
-1
0
0
-1
0
-1
0
0
-1
0
0
1
0
0
0
1
1
-1
-1
1
0
0
0
-1
0
-1
-1
1
0
0
0
-1
0
1
0
0
0
0
0
1
0
0
0
0
0
1
0
0
1
1
0
0
0
0
-1
0
0
0
1
0
1
0
0
1
-1
0
0
0
0
0
-1
1
0
-1
0
0
-1
0
1
0
-1
0
0
1
-1
1
1
1
0
0
1
1
0
0
0
0
0
0
0
-1
0
1
0
0
0
0
0
0
-1
0
1
0
0
0
1
0
0
0

INFO:root: Solution for N=5: w=9 (bloat=80%)


0
0
0
0
0
-1
0
0
0
0
1
-1
0
0
1
0
-1
0
0
1
-1
0
0
0
1
0
-1
0
-1
0
-1
0
0
0
0
0
0
0
1
0
0
0
-1
0
0
0
1
0
0
1
0
0
1
0
1
0
0
0
0
0
0
0
1
0
-1
0
0
0
0
0
-1
0
0
1
0
1
-1
-1
-1
-1
-1
0
0
1
1
0
0
0
-1
0
0
1
0
0
0
0
0
0
1
1
0
1
0
0
0
0
0
1
1
1
0
0
0
0
0
0
0
0
-1
0
0
1
0
0
0
0
0
0
0
-1
1
0
0
0
2
0
0
0
0
-1
0
-1
0
0
0
0
1
0
0
0
0
0
0
0
0
-1
-1
-1
0
-1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
-1
0
0
0
1
-1
-1
0
0
0
0
0
0
0
-1
0
1
0
0
0
0
0
0
0
0
-1
0
0
0
0
0
0
0
0
0
-1
-1
0
0
0
1
1
0
1
0
0
1
-1
0
1
0
-1
0
-1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-1
0
1
0
0
0
0
-1
0
0
0
0
0
0
0
1
0
0
1
-1
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
-1
0
0
0
-2
1
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
-1
0
-1
0
0
0
1
0
0
1
-1
0
0
0
0
0
-1
0
0
1
0
1
0
-1
0
0
0
0
1
0
0
0
0
0
-1
0
0
0
0
0
1
0
0
-1
0
0
1
1
-1
1
0
0
1
0
0
0
0
0
0
-1
0
0
1
0
1
-1
0
0
0
1
1
0
0
0
0
-1
-1
0
0
1
0
0
-1
0
0
0
-1
0
0
0
0
0
-1
0
0
0
0
0
0
0
0
0
-1
-1
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
1
1
-1
0
0
-1
0
1
1
-1
0
0
1
0
0
0
0
0
0
1
0
0
-1
0
0
0
0
0
0
0
0
-1
0
1
0
0
1
0
1
0
0
0
0
1
0
0
0
0
0

INFO:root: Solution for N=10: w=18 (bloat=80%)


0
0
0
0
0
0
0
0
0
0
-1
1
1
0
0
0
0
0
0
0
0
1
0
0
0
0
-1
0
0
1
0
-1
-1
0
0
0
1
0
0
1
1
1
0
-1
0
0
0
0
0
0
0
0
0
-1
0
1
0
0
0
0
0
-1
0
-1
0
0
0
0
1
-1
-1
0
0
-1
0
0
0
0
0
-1
0
1
0
-1
0
0
0
0
0
0
0
0
-1
-1
-1
0
0
1
0
-1
0
0
0
-1
-1
0
-1
1
1
-1
0
1
0
1
1
0
-1
1
0
0
-1
1
0
0
0
0
0
0
0
0
-1
0
0
0
1
0
0
0
0
0
0
0
0
1
-1
1
1
1
0
1
0
0
1
1
0
0
0
0
1
0
1
0
0
1
0
-1
0
-1
0
0
-1
-1
1
0
0
0
0
1
0
1
0
0
-1
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
-1
-1
0
0
0
0
-1
0
-1
0
1
0
0
0
0
1
0
0
1
0
0
0
0
0
1
0
0
1
-1
-1
0
0
-1
0
0
0
0
0
1
0
0
0
0
1
0
1
0
0
0
0
0
0
1
-1
0
0
0
0
0
1
0
0
0
0
0
0
-1
-1
0
0
-1
0
-1
-1
0
1
-1
0
0
0
0
0
0
-1
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
1
-1
0
0
-1
-1
1
0
0
0
1
0
0
0
-1
0
1
-1
0
0
1
0
0
0
-1
1
0
0
0
0
0
0
1
0
0
0
-1
0
0
0
0
-1
0
0
0
0
0
-1
0
1
0
0
1
0
0
-1
0
0
0
1
0
0
0
0
0
0
-1
0
-2
0
0
1
-1
0
0
0
-1
0
1
0
-1
0
0
-1
1
1
0
0
-1
-1
0
-1
0
0
0
1
0
1
-1
0
0
0
0
0
-1
0
0
-1
0
1
0
0
0
1
1
1
0
-1
0
0
0
1
0
1
0
1
0
-1
0
0
0
0
-1
-1
0
1
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
1
0
0
-1
-1
-1
-1
0
0
-1
1


INFO:root: Solution for N=20: w=93 (bloat=365%)


-1
0
0
0
0
0
1
0
0
1
-1
0
0
0
0
0
0
0
0
-1
0
0
0
0
0
0
0
0
0
0
0
-1
-1
0
0
0
0
0
1
0
0
0
-1
0
1
0
0
0
0
-1
0
0
1
1
0
0
-1
1
-1
0
1
1
1
-1
0
0
1
0
-1
1
0
1
0
0
0
1
1
0
0
0
0
0
0
0
0
1
1
1
0
0
0
0
-1
0
-1
0
1
-1
-1
0
0
0
0
0
0
-1
0
-1
1
-1
-1
0
0
0
1
0
0
0
0
-1
0
0
0
0
0
0
-1
-1
1
1
0
0
0
-1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-1
0
0
1
0
-1
0
0
0
0
0
0
0
1
0
1
1
1
0
0
0
1
0
0
-1
-1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-1
0
0
0
0
-1
0
0
0
1
-1
0
0
1
0
0
-1
0
0
0
0
-1
0
-1
-1
0
0
0
0
0
0
0
-1
-1
0
0
0
0
-1
0
0
0
-1
-1
0
0
0
0
0
1
0
0
1
-1
0
-1
0
1
1
-1
-1
0
0
0
0
1
1
0
0
0
0
0
0
1
0
1
0
0
-1
1
0
0
-1
0
1
-1
0
-1
0
0
0
0
0
0
0
-1
1
0
0
0
0
1
0
0
0
1
0
0
0
0
-1
0
0
0
0
0
-1
0
1
0
0
0
0
0
0
0
-1
0
-1
-1
0
0
0
-1
-1
0
-1
-1
-1
0
0
1
1
0
0
0
-1
0
0
0
-1
0
-1
0
0
0
0
0
0
0
0
0
0
0
-1
0
0
0
0
0
1
0
0
0
0
-1
0
1
0
1
0
0
0
0
-1
-1
0
1
0
0
1
0
0
0
0
1
0
1
0
0
1
0
1
1
0
-1
1
0
0
0
0
0
0
0
0
-1
0
0
0
-1
1
0
0
0
0
0
0
0
0
0
0
0
-1
0
0
1
0
0
0
-1
0
0
-1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-1
0
1
-1
0
0
-

KeyboardInterrupt: 